Ada plot graph di tiap function def(self,ecg,time,fs)

In [2]:
from matplotlib import pyplot as plt

class pan_tompkins_qrs():

    def band_pass_filter(self,ecg,time,fs):

        x = ecg
        y = x.copy
        y_filtered = None

        #Low Pass Filter
        #y(n) = 2y(n-1) - y(n-2) + x(n) - 2x(n-6) + x(n-12)
        for n in range(len(x)):
            y[n] = x[n]

            if (n >= 1):
                y[n] += 2*y[n-1]
            if (n >= 2):
                y[n] -= y[n-2]
            if (n >= 6):
                y[n] -= 2*x[n-6]
            if (n >= 12):
                y[n] += x[n-12]
        
        y_filtered = y.copy()

        #High Pass Filter
        #y(n) = 32x(n-16) - y(n-1) - x(n) + x(n-32)
        for n in range(len(x)):
            y_filtered[n] = -1*x[n]

            if (n >= 1):
                y_filtered[n] -= y_filtered[n-1]
            if (n >= 16):
                y_filtered[n] += 32*x[n-16]
            if(n >= 32):
                y_filtered[n] += x[n-32]
        
        
        ###Not sure what is this part for, It says its for normalization###
        max_val = max(max(y_filtered), -min(y_filtered))
        y_filtered = y_filtered/max_val
        ###Not sure what is this part for, It says its for normalization###


        ### Additional to show the Filtered signal###
        fig = plt.figure()        
        plt.plot(time/fs, y_filtered)
        plt.title('Band Pass Filtered ECG')
        ### Additional to show the Filtered signal###

        return y_filtered, fig
    
    def derivative(self,ecg,time,fs):

        x = ecg
        y_derived = x.copy
        
        #Derivative Filter
        #y(n) = [-x(n-2) - 2x(n-1) + 2x(n+1) + x(n+2)]/8
        for n in range(len(x)):
            y_derived[n] = 0

            if (n >= 1):
                y_derived[n] -= 2*x[n-1]
            if (n >= 2):
                y_derived[n] -= x[n-2]
            
            ###Not sure what this part is for###
            if (n >= 2 and n <= len(x)-2):
                y_derived[n] += 2*x[n+1]
            if (n >= 2 and n<= len(x)-3):
                y_derived[n] += x[n+2]
            ###Not sure what this part is for###

            ###Not sure why does it has to be times to "fs"###
            y_derived[n] = (x[n]*fs)/8
            ###Not sure why does it has to be times to "fs"###
        
        ### Additional to show the Filtered signal###
        fig = plt.figure()        
        plt.plot(time/fs, y_derived)
        plt.title('Derivative ECG')
        ### Additional to show the Filtered signal###

        return y_derived, fig
    
    def squaring(self,ecg,time,fs):
        
        x = ecg
        y_squared = x.copy()

        for n in range(len(x)):
            y_squared[n] = x[n]**2
        
        ### Additional to show the Filtered signal###
        fig = plt.figure()        
        plt.plot(time/fs, y_squared)
        plt.title('Squared ECG')
        ### Additional to show the Filtered signal###

        return y_squared, fig
    
    def moving_window_integration(self,ecg,time,fs):
        # Initialize result and window size for integration
        x = ecg
        y = x.copy()
        win_size = round(0.150 * fs)
        sum = 0

        # Calculate the sum for the first N terms
        for j in range(win_size):
            sum += x[j]/win_size
            y[j] = sum
        
        # Apply the moving window integration using the equation given
        for n in range(win_size,len(x)):  
            sum += x[n]/win_size
            sum -= x[n-win_size]/win_size
            y[n] = sum

        ### Additional to show the Filtered signal###
        fig = plt.figure()        
        plt.plot(time/fs, y)
        plt.title('Moving Window Integrated ECG')
        ### Additional to show the Filtered signal###

        return y, fig

    def solve(self,ecg,time,fs):
        '''
        Solver, Combines all the above functions
        :param signal: input signal
        :return: prcoessed signal

        Methodology/Explaination:
        The peak detection algorithm works on the moving window and bandpass
        filtered signal. So the input signal is first bandpassed, then the
        output of the bandpass filter is given to the derivative function and
        the result is squared. Finally the output of the squaring function
        is given to the moving window integration function and returned. 
        '''
        x = ecg

        # Convert the input signal into numpy array
        input_signal = x.iloc[:,1].to_numpy()

        # Bandpass Filter
        global bpass
        bpass = self.band_pass_filter(input_signal.copy())

        # Derivative Function
        global der
        der = self.derivative(bpass.copy())

        # Squaring Function
        global sqr
        sqr = self.squaring(der.copy())

        # Moving Window Integration Function
        global mwin
        mwin = self.moving_window_integration(sqr.copy())

        return mwin
        

Gk ada plot graph di tiap function def(self,ecg), fs di define di function yg butuh

In [1]:
from matplotlib import pyplot as plt
import numpy as np

class pan_tompkins_qrs():

    def bpf(self,x):
        
        
        
        y_filtered = None

        #Results will be delayed 16 Samples
        y = x.copy()

        #Low Pass Filter
        #y(n) = 2y(n-1) - y(n-2) + x(n) - 2x(n-6) + x(n-12)
        for n in range(len(x)):
            y[n] = x[n]

            if (n >= 1):
                y[n] += 2*y[n-1]
            if (n >= 2):
                y[n] -= y[n-2]
            if (n >= 6):
                y[n] -= 2*x[n-6]
            if (n >= 12):
                y[n] += x[n-12]
        
        y_filtered = y.copy()

        #High Pass Filter
        #y(n) = 32x(n-16) - y(n-1) - x(n) + x(n-32)
        for n in range(len(x)):
            y_filtered[n] = -1*y[n]

            if (n >= 1):
                y_filtered[n] -= y_filtered[n-1]
            if (n >= 16):
                y_filtered[n] += 32*y[n-16]
            if(n >= 32):
                y_filtered[n] += y[n-32]
        
        #Normalization
        max_val = max(max(y_filtered), -min(y_filtered))
        y_filtered = y_filtered/max_val

        return y_filtered
    
    def derivative(self,x):
        
        #Results will be delayed for 2 samples
        fs = 1000
        y_derived = x.copy()
        
        #Derivative Filter
        #y(n) = [-x(n-2) - 2x(n-1) + 2x(n+1) + x(n+2)]/8
        for n in range(len(x)):
            y_derived[n] = 0

            if (n >= 1):
                y_derived[n] -= 2*x[n-1]
            if (n >= 2):
                y_derived[n] -= x[n-2]
            
            ###Not sure what this part is for###
            if (n >= 2 and n <= len(x)-2):
                y_derived[n] += 2*x[n+1]
            if (n >= 2 and n<= len(x)-3):
                y_derived[n] += x[n+2]
            ###Not sure what this part is for###

            ###Not sure why does it has to be times to "fs"###
            y_derived[n] = (y_derived[n]*fs)/8
            ###Not sure why does it has to be times to "fs"###
        
        return y_derived
    
    def squaring(self,x):
        
        y_squared = x.copy()

        for n in range(len(x)):
            y_squared[n] = x[n]**2

        return y_squared
    
    def moving_window_integration(self,x,fs):
        # Initialize result and window size for integration
        y = x.copy()
        win_size = round(0.150 * fs)
        sum = 0

        # Calculate the sum for the first N terms
        for j in range(win_size):
            sum += x[j]/win_size
            y[j] = sum
        
        # Apply the moving window integration using the equation given
        for n in range(win_size,len(x)):  
            sum += x[n]/win_size
            sum -= x[n-win_size]/win_size
            y[n] = sum

        return y

    def solve(self,x):

        # Convert the input signal into numpy array
        input_signal = x.copy()

        # Bandpass Filter
        global bpass
        bpass = self.bpf(input_signal.copy())

        # Derivative Function
        global der
        der = self.derivative(bpass.copy())

        # Squaring Function
        global sqr
        sqr = self.squaring(der.copy())

        # Moving Window Integration Function
        global mwin
        mwin = self.moving_window_integration(sqr.copy())

        return mwin